In [1]:
import numpy as np
import pickle
import glob
from numpy.lib.function_base import average
from numpy import load
from numpy import save
from tqdm import tqdm


from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

path = "D:\\tuc\\exam10\\Thesis\\Dataset\\CMU_clips_no_zip.npy"

X = load(path, allow_pickle=True)
print(X.shape)


(1073, 3198, 31, 6)
